# Continued Analysis


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

from typing import Optional
from enrich.style import STYLES
from rich.theme import Theme
from rich.console import Console

from ClimRR import (
    set_plot_style,
    get_logger,
    DEFAULT_CRS,
)
from ClimRR.data import (
    load_shapefile,
    load_csvs,
    load_counties,
    load_chicago_data
)

set_plot_style()
theme = Theme(STYLES)
log = get_logger('ClimRR')
console = Console(theme=theme, log_path=False, markup=True)

In [ ]:
shape = load_shapefile()
data = load_csvs(shape)
chidata = load_chicago_data(shape.crs)

In [ ]:
counties = load_counties(shape.crs)

In [ ]:
illinois = counties[counties["ste_name"] == "['Illinois']"]
wind_il = data["WindSpeed"].overlay(illinois, how='intersection')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
wind_il.plot(ax=ax, column='hist', legend=True)
ax.set_axis_off()
plt.show()

In [ ]:
console.log("\n".join(list(data.keys())))

In [ ]:
def get_state(
        state: str,
        counties: Optional[gpd.GeoDataFrame] = None,
) -> gpd.GeoDataFrame:
    counties = load_counties(DEFAULT_CRS) if counties is None else counties
    assert counties is not None and isinstance(counties, gpd.GeoDataFrame)
    assert counties.crs == DEFAULT_CRS
    return counties[counties["ste_name"] == f"['{state}']"]


def get_variable_for_state(
        data: gpd.GeoDataFrame,
        state: gpd.GeoDataFrame,
) -> gpd.GeoDataFrame:
    """Returns `data.overlay(state, how='intersection')`"""
    return data.overlay(state, how='intersection')

In [ ]:
california = get_state('California', counties=counties)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax = california.plot(ax=ax)
ax.set_axis_off()

In [ ]:
cali_temp_max = get_variable_for_state(
    data["SeasonalTemperatureMaximum"],
    california
)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7), nrows=2, ncols=3, sharey='row', sharex='col')
ax = ax.flatten()
ax0 = cali_temp_max.plot(ax=ax[0], column='hist_winter', legend=True)
ax0.set_title('Winter [Historical]')
ax0.set_axis_off()
ax1 = cali_temp_max.plot(ax=ax[1], column='rcp85_midc_winter', legend=True)
ax1.set_axis_off()
ax1.set_title('Winter [RCP85: Mid-Century]')
ax2 = cali_temp_max.plot(ax=ax[2], column='rcp85_endc_winter', legend=True)
ax2.set_axis_off()
ax2.set_title('Winter [RCP85: End-Century]')

ax3 = cali_temp_max.plot(ax=ax[3], column='hist_summer', legend=True)
ax3.set_title('Summer [Historical]')
ax3.set_axis_off()
ax4 = cali_temp_max.plot(ax=ax[4], column='rcp85_mid_summer', legend=True)
ax4.set_axis_off()
ax4.set_title('Summer [RCP85: Mid-Century]')
ax5 = cali_temp_max.plot(ax=ax[5], column='rcp85_end_summer', legend=True)
ax5.set_axis_off()
ax5.set_title('Summer [RCP85: End-Century]')
plt.tight_layout()